In [1]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import imutils
import sys
import numpy
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
numpy.set_printoptions(threshold=sys.maxsize)

%run commons.py

4.1.1


# load Naive Bayes

In [2]:
filename = 'naive_bayes_model.sav'
gnb = pickle.load(open(filename, 'rb'))

filename_blurred = 'blurred_naive_bayes_model.sav'
gnb_blurred = pickle.load(open(filename_blurred, 'rb'))

## Define pipeline functions

In [3]:
def show(img):
 cv2.imshow('image',img)
 cv2.waitKey(0)
 cv2.destroyAllWindows()

In [4]:
def detect(img, model):
    w,h,ch = img.shape
    f = features(img, channels, False)
    pred = model.predict(f)
    img_pred = np.reshape(pred, (w,h,1))
    return np.logical_not(img_pred)

In [5]:
def median_filter(img):
    kernel = np.ones((5,5),np.float32)/25
    img = cv2.filter2D(img,-1,kernel)
    return cv2.threshold(img,0.5,1.0,cv2.THRESH_BINARY)[1]

In [6]:
def rotate(img, angle):
    return  1-imutils.rotate(1-img, angle)

In [7]:
def odd(num):
    return (num % 3) == 0

In [8]:
def find_pallet_by_height(img, mask_height, max_row_ind, min_perc):
    results=[]
    mask_height = mask_height
    mask_width = int(mask_height * 5.556)
    mask_size = mask_height * mask_width
    hole_height = int(mask_height * 0.694)
    hole_width = int(mask_height * 1.58)
    hole_size = hole_height * hole_width
    hole_1_x = int(mask_height * 0.694)
    hole_1_y = int(mask_height * 0.306)
    hole_2_x = int(mask_height * 3.281)
    hole_2_y = hole_1_y
    inverse_img = np.logical_not(img)
    img_height, img_width = inverse_img.shape
    for index, _ in np.ndenumerate(inverse_img):
        x,y = index
        y+=max_row_ind
        if odd(x) and odd(y) and y+mask_height < img_height and x+mask_width < img_width:
            frame_mask = inverse_img[y:y+mask_height, x:x+mask_width]
            hole_1_y_ = y+hole_1_y
            hole_2_y_ = y+hole_2_y
            hole_1_x_ = x+hole_1_x
            hole_2_x_ = x+hole_2_x
            hole_1_mask = inverse_img[hole_1_y_:hole_1_y_+hole_height, hole_1_x_:hole_1_x_+hole_width]
            hole_2_mask = inverse_img[hole_2_y_:hole_2_y_+hole_height, hole_2_x_:hole_2_x_+hole_width]
            frame_mask_perc = np.sum(frame_mask) / mask_size
            hole_1_mask_perc = np.sum(hole_1_mask) / hole_size
            hole_2_mask_perc = np.sum(hole_2_mask) / hole_size
            perc = frame_mask_perc - 2*hole_1_mask_perc - 2*hole_2_mask_perc
            result=(frame_mask, perc, (x,y), mask_height)
            if perc >= min_perc:
                results.append(result)
    return results

def mean_of_result_centers(results):
    centers=[np.array(x[2]) for x in results]
    mean = tuple(np.mean(centers, axis=0))
    return mean

def resolve_redundancy(results):
    bins=dict()
    acceptance_thr = 50
    for result in results:
        (frame_mask, perc, center, mask_height) = result
        matching_bins = [bin_key for bin_key in bins.keys() if dist(center, bin_key)<acceptance_thr]
        if matching_bins:
            matching_bin=matching_bins[0]
            all_bin_results = bins.pop(matching_bin)+[result]
            mean_center=mean_of_result_centers(all_bin_results)
            bins[mean_center] = all_bin_results
        else:
            bins[center]=[result]
    results=[max(bin_results, key=lambda x: x[1]) for bin_results in bins.values()]
    return results

def get_optim_row(img, margin, visualize):
    hist = [1.0 - np.mean(row) for row in img]
    max_row_ind = np.argmax(hist)
    optim_row_ind = max(0,max_row_ind-margin)
    if visualize:
        rows = np.arange(len(hist))
        print("optim_row_ind",optim_row_ind)
        plt.bar(rows, hist)
        plt.show()
    return optim_row_ind

def find_pallet(img, min_height, max_height, step, min_perc, dense_row_margin):
    results=[]
    most_dense_row=get_optim_row(img, dense_row_margin, visualize=False)
    for mask_height in range(min_height, max_height, step):
        mask_height_results = find_pallet_by_height(img, mask_height, most_dense_row, min_perc)
        results.extend(mask_height_results)
    return resolve_redundancy(results)

def dist(a,b): #fixme copy from evaluation.ipynb
    return np.linalg.norm(np.array(a)-np.array(b))

def draw_pallets(img_full, results):
    color = (0,255,0)
    for result in results:
        (frame_mask, perc, (x,y), mask_height)=result 
        mask_width = int(mask_height * 5.556)
        img_full[y:y+mask_height, x:x+1]=color
        img_full[y:y+mask_height, x+mask_width-1:x+mask_width]=color
        img_full[y:y+1, x:x+mask_width]=color
        img_full[y+mask_height-1:y+mask_height, x:x+mask_width]=color
    return img_full

def calculate_centers(results):
    centers=[]
    for result in results:
        (frame_mask, perc, (x,y), mask_height)=result 
        print(perc)
        mask_width = int(mask_height * 5.556)
        x = x + mask_width/2
        y = y + mask_height/2
        center = (x,y)
        centers.append(center)
    return centers


def denoising(img, with_save, kernel):
    img = 1-img
    kernel = np.ones(kernel, np.uint8) 
    img_erosion = cv2.erode(img, kernel, iterations=1)
    if with_save:
        save(1-img_erosion, "img_erosion.jpg")
    img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
    if with_save:
        save(1-img_dilation, "img_dilation.jpg")
    return 1-img_dilation

def filling(img, with_save, kernel):
    img = 1-img
    kernel = np.ones(kernel, np.uint8) 
    img_dilation = cv2.dilate(img, kernel, iterations=1)
    if with_save:
        save(1-img_dilation, "img_dilation_2.jpg")
    img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
    if with_save:
        save(1-img_erosion, "img_erosion_2.jpg")
    return 1-img_erosion

In [9]:
def save(img, file, binary=True):
    res = cv2.imwrite(file, img * 255 if binary else img)
    print("saved" if res else "save error", file)

# Set parameters

In [19]:
MIN_MASK_WIDTH = 25
MAX_MASK_WIDTH = 100
MASK_WIDTH_STEP = 3
CONF_TH = 0.22
KERNEL=(7,7)
DENSE_ROW_MARGIN=30
ANGLE=0

## Run pipeline and save results

In [23]:
img_full = cv2.imread('r_1_39.jpg')
img_blurred = cv2.medianBlur(img_full, 7)

img_classified = detect(img_full, gnb).astype('float32')
save(img_classified, "img_classified.jpg")

#img_filtered = median_filter(img_classified)
#save(img_filtered, "img_filtered.jpg")
img_denoised = denoising(img_classified, True, KERNEL)
img_filled = filling(img_denoised, True, KERNEL)

img_rotated = rotate(img_filled, ANGLE)
save(img_rotated, "img_rotated.jpg")

results = find_pallet(img_rotated, MIN_MASK_WIDTH, MAX_MASK_WIDTH, MASK_WIDTH_STEP, CONF_TH, DENSE_ROW_MARGIN)
#save(img_pallet, "img_pallet.jpg")
print("Found pallets count:", len(results))

img_full_marked = draw_pallets(img_full, results)
save(img_full_marked, "img_full_marked.jpg", False)
centers = calculate_centers(results)
print("centers", centers)

saved img_classified.jpg
saved img_erosion.jpg
saved img_dilation.jpg
saved img_dilation_2.jpg
saved img_erosion_2.jpg
saved img_rotated.jpg
Found pallets count: 0
saved img_full_marked.jpg
centers []


# Pipeline as function

In [20]:
def localise(path,filename):
    img_full = cv2.imread(path+filename)
    #img_blurred = cv2.medianBlur(img_full, 7)
    img_classified = detect(img_full, gnb).astype('float32')
    img_denoised = denoising(img_classified, False, KERNEL)
    #img_filled = filling(img_denoised, False, KERNEL)
    #img_filtered = median_filter(img_classified)
    img_rotated = rotate(img_filled, ANGLE)
    results = find_pallet(img_rotated, MIN_MASK_WIDTH, MAX_MASK_WIDTH, MASK_WIDTH_STEP, CONF_TH, DENSE_ROW_MARGIN)
    img_full_marked = draw_pallets(img_full, results)
    save(img_full_marked, path+"localised/"+filename, False)
    return calculate_centers(results)

In [21]:
def localize_all():
    centers = dict()
    path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/"
    filenames = [f for f in sorted(listdir(path)) if isfile(join(path, f))]
    for i in tqdm(range(len(filenames))):
        filename = filenames[i]
        print(path, filename)
        centers[filename]=localise(path,filename)
    np.save(path+"eval/pred_centers", centers)

In [22]:
localize_all()

  0%|          | 0/100 [00:00<?, ?it/s]

/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_0.jpg


  1%|          | 1/100 [00:12<21:24, 12.97s/it]

saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_1_0.jpg
0.2821808921129804
0.2941747341597304
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_1.jpg


  2%|▏         | 2/100 [00:25<21:05, 12.92s/it]

...

saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_2_7.jpg
0.22770804642927406
0.2290793441163648
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_2_8.jpg


 99%|█████████▉| 99/100 [19:55<00:11, 11.90s/it]

saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_2_8.jpg
0.2807984036337177
0.24297411733693597
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_2_9.jpg


100%|██████████| 100/100 [20:07<00:00, 12.07s/it]

saved /home/maciej/repos/pallet-recognition/data/jpeg_marked/localised/r_2_9.jpg
0.2821753797984698
0.32854386671061475
